In [59]:
import pandas as pd
import numpy as np

file_name = "X03_경찰청_범죄 발생 지역별 통계_20231231.csv"
df_original = pd.read_csv(f"./{file_name}",encoding="euc-kr")
df = df_original.copy()
df.head()

,범죄대분류,범죄중분류,서울종로구,서울중구,서울용산구,서울성동구,서울광진구,서울동대문구,서울중랑구,서울성북구,...,외국러시아,외국튀르키예,외국중국,외국일본,외국필리핀,외국베트남,외국태국,외국말레이시아,외국인도네시아,외국기타국가
0,강력범죄,살인기수,2,0,1,1,0,2,4,1,...,0,0,0,0,1,0,0,0,0,0
1,강력범죄,살인미수등,1,3,2,1,2,3,2,0,...,1,0,1,0,0,0,0,0,0,1
2,강력범죄,강도,5,6,3,2,5,4,2,1,...,0,0,0,0,3,0,0,0,0,0
3,강력범죄,강간,25,30,45,22,42,36,40,25,...,0,0,3,1,3,1,1,1,0,3
4,강력범죄,유사강간,10,4,4,6,9,5,9,7,...,0,0,0,0,1,0,0,1,0,1


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Columns: 249 entries, 범죄대분류 to 외국기타국가
dtypes: int64(247), object(2)
memory usage: 74.1+ KB


In [61]:
df_filtered = df.filter(regex='서울|범죄')
df_filtered.head()

,범죄대분류,범죄중분류,서울종로구,서울중구,서울용산구,서울성동구,서울광진구,서울동대문구,서울중랑구,서울성북구,...,서울강서구,서울구로구,서울금천구,서울영등포구,서울동작구,서울관악구,서울서초구,서울강남구,서울송파구,서울강동구
0,강력범죄,살인기수,2,0,1,1,0,2,4,1,...,0,0,1,6,0,3,1,2,2,1
1,강력범죄,살인미수등,1,3,2,1,2,3,2,0,...,1,12,3,6,3,14,6,12,6,3
2,강력범죄,강도,5,6,3,2,5,4,2,1,...,6,7,5,9,3,10,9,19,9,5
3,강력범죄,강간,25,30,45,22,42,36,40,25,...,77,34,25,51,18,78,55,181,64,34
4,강력범죄,유사강간,10,4,4,6,9,5,9,7,...,12,7,5,12,2,20,9,38,7,6


In [62]:
df_filtered['범죄중분류'].unique()

array(['살인기수', '살인미수등', '강도', '강간', '유사강간', '강제추행', '기타 강간 강제추행등', '방화',
       '절도범죄', '상해', '폭행', '체포감금', '협박', '약취 유인', '폭력행위등', '공갈', '손괴',
       '직무유기', '직권남용', '증수뢰', '통화', '문서 인장', '유가증권인지', '사기', '횡령', '배임',
       '성풍속범죄', '도박범죄', '특별경제범죄', '마약범죄', '보건범죄', '환경범죄', '교통범죄', '노동범죄',
       '안보범죄', '선거범죄', '병역범죄', '기타범죄'], dtype=object)

'범죄중분류'중 여성 안전도에 직접적이고 위협적인 범죄를 우선적으로 선택

In [71]:
target_crimes = ['강간', '유사강간', '강제추행', '기타 강간 강제추행등', '성풍속범죄']

regex_pattern = '|'.join(target_crimes)
df_sexual= df_filtered[df_filtered['범죄중분류'].isin(target_crimes)]

df_sexual

,범죄대분류,범죄중분류,서울종로구,서울중구,서울용산구,서울성동구,서울광진구,서울동대문구,서울중랑구,서울성북구,...,서울강서구,서울구로구,서울금천구,서울영등포구,서울동작구,서울관악구,서울서초구,서울강남구,서울송파구,서울강동구
3,강력범죄,강간,25,30,45,22,42,36,40,25,...,77,34,25,51,18,78,55,181,64,34
4,강력범죄,유사강간,10,4,4,6,9,5,9,7,...,12,7,5,12,2,20,9,38,7,6
5,강력범죄,강제추행,204,182,195,92,188,107,118,105,...,203,150,93,245,176,220,340,464,233,99
6,강력범죄,기타 강간 강제추행등,0,0,0,0,0,1,0,2,...,2,0,2,0,0,3,2,2,1,0
26,풍속범죄,성풍속범죄,123,108,127,131,173,140,122,173,...,262,137,85,195,159,315,223,414,252,155


In [72]:
# 1. '범죄중분류' 컬럼을 인덱스로 설정합니다. (컬럼의 값이 행 이름이 됨)
df_indexed = df_sexual.set_index('범죄중분류')

# 2. 불필요한 '범죄대분류' 컬럼을 삭제합니다.
df_indexed = df_indexed.drop('범죄대분류', axis=1) # (이미 이전 단계에서 삭제되었다면 생략)

# 3. 행과 열을 바꿉니다 (전치: .T).
#    * 새로운 인덱스(행) : 지역구 이름 (예: 서울종로구)
#    * 새로운 컬럼(열) : 범죄중분류 값 (예: 강간, 유사강간)
df_final = df_indexed.T

# 4. 컬럼 이름에서 '서울' 단어를 제거하여 지역구 이름만 남깁니다.
df_final.columns.name = None # 이전 인덱스 이름(범죄중분류) 제거
df_final.index = df_final.index.str.replace('서울', '')

df_final

,강간,유사강간,강제추행,기타 강간 강제추행등,성풍속범죄
종로구,25,10,204,0,123
중구,30,4,182,0,108
용산구,45,4,195,0,127
성동구,22,6,92,0,131
광진구,42,9,188,0,173
동대문구,36,5,107,1,140
중랑구,40,9,118,0,122
성북구,25,7,105,2,173
강북구,21,9,108,1,122
도봉구,10,4,74,0,95


In [75]:
# 1️⃣ 지역구를 인덱스에서 컬럼으로 옮기기
df_final = df_final.reset_index()
df_final.rename(columns={'index': '지역구'}, inplace=True)

# 2️⃣ 각 행의 합계(‘지역구’ 제외)를 계산해 ‘합계’ 컬럼 추가
df_final['합계'] = df_final.iloc[:, 1:].sum(axis=1)

# 3️⃣ 지역구 기준으로 오름차순 정렬 (선택사항)
df_final = df_final.sort_values(by='지역구').reset_index(drop=True)

# 결과 확인
df_final


,지역구,강간,유사강간,강제추행,기타 강간 강제추행등,성풍속범죄,합계
0,강남구,181,38,464,2,414,1099
1,강동구,34,6,99,0,155,294
2,강북구,21,9,108,1,122,261
3,강서구,77,12,203,2,262,556
4,관악구,78,20,220,3,315,636
5,광진구,42,9,188,0,173,412
6,구로구,34,7,150,0,137,328
7,금천구,25,5,93,2,85,210
8,노원구,41,2,110,1,205,359
9,도봉구,10,4,74,0,95,183


In [ ]:
# csv파일로 저장
file_name = 'X03_전처리_범죄발생_건수_20231231기준.csv'

# DataFrame을 CSV 파일로 저장합니다.
df_final.to_csv(file_name, encoding='utf-8')

print(f"DataFrame이 '{file_name}' 이름으로 저장되었습니다.")

DataFrame이 'X03_전처리_경찰청_범죄 발생 지역별 통계_20231231.csv' 이름으로 저장되었습니다.
